In [3]:

from django.db.models import Subquery, Max, OuterRef, F, Sum, Count
import sqlparse
from main.models import Product, SaleRecord, Channel
from main.display import table
from functools import partial

def print_sql(queryset):
    format = partial(sqlparse.format, reindent=True, keyword_case='upper')
    query = str(queryset.query)
    print(format(query))

## Outline

* SQL 前情提要
* ORM 前請提要
* SQL Compilation 
* WorkShop
    * data introduction
    * explanation
    * a small practice
* Performance Profile



## SQL


* 在要達成一樣的功能的情況下，SQL Query 通常比你自已寫程式快。
* 難度適中，但是跟一般程式的想法稍有不同
* 每家 db 可能各有強項，提供的 function 跟語法也都有不同





## ORM

* 讓你簡化程式跟程資料庫之間的橋接
* 安全性更好
* 讓你有機會可以無痛更換 database
* Query 資料時，會幫你 compile sql query
* BUT. 有些進階的 database 新功能，可能就會慢一步才支援

# How Django Compile SQL

In [12]:

queryset = SaleRecord.objects.all()[:5]
table(queryset.values())


,channel_id,count,datetime,price,product_id
0,1,143,2017-07-04 01:01:01+00:00,18,1
1,1,282,2017-10-10 01:01:01+00:00,18,1
2,1,16,2018-01-05 01:01:01+00:00,18,1
3,1,110,2018-04-06 01:01:01+00:00,18,1
4,1,23,2018-07-06 01:01:01+00:00,18,1


In [19]:
print_sql(queryset)

SELECT "main_salerecord"."id",
       "main_salerecord"."channel_id",
       "main_salerecord"."product_id",
       "main_salerecord"."price",
       "main_salerecord"."count",
       "main_salerecord"."datetime"
FROM "main_salerecord"
LIMIT 5


In [44]:

queryset = SaleRecord.objects.filter(product__cost__lte=20)
print_sql(queryset)

SELECT "main_salerecord"."id",
       "main_salerecord"."channel_id",
       "main_salerecord"."product_id",
       "main_salerecord"."price",
       "main_salerecord"."count",
       "main_salerecord"."datetime"
FROM "main_salerecord"
INNER JOIN "main_product" ON ("main_salerecord"."product_id" = "main_product"."id")
WHERE "main_product"."cost" <= 20


# Data

In [41]:
table(Product.objects.all().values())

,cost,name
0,8,王紫麵
1,18,真由味
2,26,湖遲屋
3,38,品嗑


In [42]:
table(Channel.objects.all().values())

,name
0,加熱福
1,好是多
2,拳家


In [45]:
table(SaleRecord.objects.all()[:20].values())

,channel_id,count,datetime,price,product_id
0,1,143,2017-07-04 01:01:01+00:00,18,1
1,1,282,2017-10-10 01:01:01+00:00,18,1
2,1,16,2018-01-05 01:01:01+00:00,18,1
3,1,110,2018-04-06 01:01:01+00:00,18,1
4,1,23,2018-07-06 01:01:01+00:00,18,1
5,1,78,2017-07-04 01:01:01+00:00,28,2
6,1,160,2017-10-10 01:01:01+00:00,28,2
7,1,221,2018-01-05 01:01:01+00:00,28,2
8,1,148,2018-04-06 01:01:01+00:00,28,2
9,1,78,2018-07-06 01:01:01+00:00,28,2


## 暖身 為每一筆衣料 補上更多資料

In [52]:
sales_record_queryset = SaleRecord.objects.annotate(
    
    product_name=F('product__name'),
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
    
).annotate(
    margin=F('amount') - F('cost')
)

table(sales_record_queryset.values()[:20])

## Practice 試試看如何 annotate 利潤所佔的比例

,amount,channel_id,cost,count,datetime,margin,price,product_id,product_name
0,2574,1,1144,143,2017-07-04 01:01:01+00:00,1430,18,1,王紫麵
1,5076,1,2256,282,2017-10-10 01:01:01+00:00,2820,18,1,王紫麵
2,288,1,128,16,2018-01-05 01:01:01+00:00,160,18,1,王紫麵
3,1980,1,880,110,2018-04-06 01:01:01+00:00,1100,18,1,王紫麵
4,414,1,184,23,2018-07-06 01:01:01+00:00,230,18,1,王紫麵
5,2184,1,1404,78,2017-07-04 01:01:01+00:00,780,28,2,真由味
6,4480,1,2880,160,2017-10-10 01:01:01+00:00,1600,28,2,真由味
7,6188,1,3978,221,2018-01-05 01:01:01+00:00,2210,28,2,真由味
8,4144,1,2664,148,2018-04-06 01:01:01+00:00,1480,28,2,真由味
9,2184,1,1404,78,2018-07-06 01:01:01+00:00,780,28,2,真由味


## Question 1


##  每一個產品最近一次的銷售狀況





## 先做一個產品

In [48]:
p1 = Product.objects.first()
p1_record = SaleRecord.objects.filter(product=p1)

newest = p1_record.order_by('datetime').reverse()

print p1.name
print newest.first().datetime
print newest.first().price
print newest.first().count

table(newest.values()[:1])

王紫麵
2018-07-06 01:01:01+00:00
18
23


,channel_id,count,datetime,price,product_id
0,1,23,2018-07-06 01:01:01+00:00,18,1


## 全部的產品

## Loop maybe ok.  but tedius

## 使用 Subquery 

In [58]:
sales_record_queryset = SaleRecord.objects.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sales_record_queryset = sales_record_queryset.filter(
    product=OuterRef('pk')
)

latest_record = sales_record_queryset.order_by('datetime').reverse()[:1]

products = Product.objects.all().annotate(
    latest_amount=Subquery(latest_record.values('amount')),
    latest_margin=Subquery(latest_record.values('margin')),
    latest_cost=Subquery(latest_record.values('cost')),
    latest_count=Subquery(latest_record.values('count')),
    latest_price=Subquery(latest_record.values('price')),
    channel=Subquery(latest_record.values('channel__name')),
)

table(products.values())  ## only for show up


,channel,cost,latest_amount,latest_cost,latest_count,latest_margin,latest_price,name
0,加熱福,8,414,184,23,230,18,王紫麵
1,加熱福,18,2184,1404,78,780,28,真由味
2,加熱福,26,7776,5616,216,2160,36,湖遲屋
3,加熱福,38,3888,3078,81,810,48,品嗑


In [59]:
print_sql(products)

SELECT "main_product"."id",
       "main_product"."name",
       "main_product"."cost",

  (SELECT U0."count"
   FROM "main_salerecord" U0
   INNER JOIN "main_product" U1 ON (U0."product_id" = U1."id")
   WHERE U0."product_id" = ("main_product"."id")
   ORDER BY U0."datetime" DESC
   LIMIT 1) AS "latest_count",

  (SELECT U0."price"
   FROM "main_salerecord" U0
   INNER JOIN "main_product" U1 ON (U0."product_id" = U1."id")
   WHERE U0."product_id" = ("main_product"."id")
   ORDER BY U0."datetime" DESC
   LIMIT 1) AS "latest_price",

  (SELECT (U1."cost" * U0."count") AS "cost"
   FROM "main_salerecord" U0
   INNER JOIN "main_product" U1 ON (U0."product_id" = U1."id")
   WHERE U0."product_id" = ("main_product"."id")
   ORDER BY U0."datetime" DESC
   LIMIT 1) AS "latest_cost",

  (SELECT ((U0."price" * U0."count") - (U1."cost" * U0."count")) AS "margin"
   FROM "main_salerecord" U0
   INNER JOIN "main_product" U1 ON (U0."product_id" = U1."id")
   WHERE U0."product_id" = ("main_product"."

## 今年總營業額

In [60]:

sale_queryset = SaleRecord.objects.all()

sale_queryset = sale_queryset.filter(datetime__gte='2018-01-01')

# below are further reading
# from django.db.models.functions import Now, TruncYear
# sale_queryset = sale_queryset.annotate(year=TruncYear('datetime'))
# sale_queryset = sale_queryset.filter(year__gte=TruncYear(Now()))

sale_queryset = sale_queryset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sale_queryset.aggregate(
    total_amount=Sum('amount'),
    total_margin=Sum('margin'),
    total_cost=Sum('cost')
)



/private/tmp/orm-workshop/venv/lib/python2.7/site-packages/django/db/models/fields/__init__.py:1451: RuntimeWarning: DateTimeField SaleRecord.datetime received a naive datetime (2018-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)


{'total_amount': 150951, 'total_cost': 119616, 'total_margin': 31335}

### 每一月份的營業額

In [51]:
from datetime import datetime
from django.db.models.functions import Trunc

sale_questset = SaleRecord.objects.all()

sale_questset = sale_questset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost'),
    month=Trunc('datetime', 'month'),
)

sale_questset = sale_questset.annotate(
    total_amount=Sum('amount'),
    total_cost=Sum('cost'),
    total_margin=Sum('margin'),
)

sale_questset = sale_questset.values(
    'product__name', 
    'month', 
    'total_margin',
    'total_amount',
    'total_cost',
).annotate(sale_count=Sum('count'))


table(sale_questset)

,month,product__name,sale_count,total_amount,total_cost,total_margin
0,2017-07-01 00:00:00+00:00,品嗑,335,16153,12730,3423
1,2017-10-01 00:00:00+00:00,品嗑,374,16702,14212,2490
2,2018-01-01 00:00:00+00:00,品嗑,722,33921,27436,6485
3,2018-04-01 00:00:00+00:00,品嗑,544,24662,20672,3990
4,2018-07-01 00:00:00+00:00,品嗑,524,23794,19912,3882
5,2017-07-01 00:00:00+00:00,湖遲屋,486,16576,12636,3940
6,2017-10-01 00:00:00+00:00,湖遲屋,514,17048,13364,3684
7,2018-01-01 00:00:00+00:00,湖遲屋,449,15020,11674,3346
8,2018-04-01 00:00:00+00:00,湖遲屋,631,22244,16406,5838
9,2018-07-01 00:00:00+00:00,湖遲屋,262,7952,6812,1140


## 各個 channel 總成績


In [14]:

sale_queryset = SaleRecord.objects.all()

channel = Channel.objects.first()

# sale_queryset = sale_queryset.filter(channel=channel)

sale_queryset = sale_queryset.filter(channel=OuterRef('pk'))

sale_queryset = sale_queryset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sale_queryset = sale_queryset.annotate(
    total_amount=Sum('amount'),
    total_cost=Sum('cost'),
    total_margin=Sum('margin'),
)

sale_queryset = sale_queryset.values(
    'channel',
    'total_amount',
    'total_cost',
    'total_margin',
).annotate(sale_count=Sum('count'))


channel_queryset = Channel.objects.all().annotate(
    
    total_amount=Subquery(sale_queryset.values('total_amount')),
    total_margin=Subquery(sale_queryset.values('total_margin')),
    total_cost=Subquery(sale_queryset.values('total_cost')),
    total_count=Subquery(sale_queryset.values('sale_count')),


)
table(channel_queryset.values())


# from django.db.models import ExpressionWrapper, FloatField
# from django.db.models.functions import Substr

# rate_expression = Substr(((F('total_margin')+0.0)/F('total_amount')*100), 1, 2)

# channel_queryset = channel_queryset.annotate(
    
#     profit_rate= rate_expression,
# )


,name,total_amount,total_cost,total_count,total_margin
0,加熱福,96040,67380,2866,28660
1,好是多,89961,83994,3217,5967
2,拳家,93977,67698,3102,26279


In [17]:
q = Channel.objects.all().filter(salerecord__channel=F('pk')).annotate(
    total_count = Sum('salerecord__count', unique=True),
).annotate(
    total_amount = F('total_count') * F('salerecord__price')
)
table(q.values('total_amount'))

,name,total_amount,total_count
0,加熱福,10332,574
1,加熱福,19180,685
2,加熱福,31824,884
3,加熱福,34704,723
4,好是多,4576,572
5,好是多,11989,631
6,好是多,19768,706
7,好是多,53628,1308
8,拳家,8327,757
9,拳家,20850,834
